In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, LearningRateScheduler

import glob
import os

# Setting seed for reproducability
#np.random.seed(1234)  
#PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM, Masking, Activation
%matplotlib inline

from data_generator import TSDataGenerator
from util import set_log_dir, rmse
from util import LRDecay

Using TensorFlow backend.


In [4]:
DATA_DIR = "./data/"
MODEL_DIR = "./model/"

### Load Data

The data used for this project is the NASA C-MAPSS Turbofan Engine Degradation Data Set https://ti.arc.nasa.gov/c/6/.  This data is model based simulated data from the Commercial Modular Aero-Propulsion System Simulation (C-MAPSS).

In [5]:
!ls {DATA_DIR}

 CMAPSSDATA.zip			    RUL_FD004.txt    test_y.csv
'Damage Propagation Modeling.pdf'   test_FD001.txt   train.csv
 readme.txt			    test_FD002.txt   train_FD001.txt
 RUL_FD001.txt			    test_FD003.txt   train_FD002.txt
 RUL_FD002.txt			    test_FD004.txt   train_FD003.txt
 RUL_FD003.txt			    test_x.csv	     train_FD004.txt


The data set is a multivariate time series. Each entry (row) reflects an operational cycle of a specific engine identified by engine id and cycle time. There are multiple entries per engine to represent different reporting times. Other columns represents different features 3 operational settings and 21 sensors:

<pre>
    1)      engine id
    2)      time, in cycles
    3)      operational setting 1
    4)      operational setting 2
    5)      operational setting 3
    6)      sensor measurement  1
    7)      sensor measurement  2
    ...
    26)     sensor measurement  21
</pre>

In [13]:
cols = ['id', 'cycle' ]

# Three operational setting columns
setting_cols = ['setting' + str(i) for i in range(1,4)]
cols.extend(setting_cols)

# Twenty one sensor columns
sensor_cols = ['s' + str(i) for i in range(1,22)]
cols.extend(sensor_cols)

sort_cols = ['id','cycle']


In [221]:
def load_data_single(path, col_names, sort_cols):
    # read data 
    df = pd.read_csv(path, sep=" ", header=None) 
    df.drop(df.columns[[26, 27]], axis=1, inplace=True)
    df.columns = col_names
    df['filename'] = os.path.splitext(os.path.basename(path))[0]
    df = df.sort_values(sort_cols)
    return df

The CMAPSS data is divided into training, test, and RUL data files. Each of these is further partitioned in 4 subsets that represents a different operational condition. The number of engines in each vary.

In [ ]:


fn_id_map = {
    "train_FD001": 1000,
    "train_FD002": 2000,
    "train_FD003": 3000,
    "train_FD004": 4000,
    "test_FD001":  5000,
    "test_FD002": 6000,
    "test_FD003": 7000,
    "test_FD004": 8000,    
}


# Filename is mapped to a condition. Map:
#       ONE (Sea Level) to 1
#       SIX to 2
#       unknown is left as 0
fn_condition_map = {
    "train_FD001": 1,
    "train_FD002": 2,
    "train_FD003": 1,
    "train_FD004": 2,
    "test_FD001":  1,
    "test_FD002":  2,
    "test_FD003":  1,
    "test_FD004":  2,    
}

In [223]:

def load_data(paths, col_names, sort_cols):
    # read data 
    df = pd.DataFrame()
    for p in paths:
        instance_df = pd.read_csv(p, sep=" ", header=None)
        instance_df.drop(instance_df.columns[[26, 27]], axis=1, inplace=True)
        instance_df.columns = col_names
        instance_df['filename'] = os.path.splitext(os.path.basename(p))[0]
        
        df = pd.concat((df, instance_df), sort=False) 

    df['condition'] = df['filename'].apply( lambda f: fn_condition_map[f])
    df['id'] = df['id'] + df['filename'].apply( lambda f: fn_id_map[f])
    df.drop(['filename'], axis=1, inplace=True)
    df = df.sort_values(sort_cols)
    return df

Read training, validation, and test data 


In [224]:
path = os.path.join(DATA_DIR, "train_FD*.txt")
all_files = glob.glob(path)

train_df = load_data(all_files, cols, sort_cols)

val_df = load_data([DATA_DIR+'test_FD001.txt'], cols, sort_cols)

In [225]:
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,condition
0,1001,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,1
1,1001,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,1
2,1001,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,1
3,1001,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,1
4,1001,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,1


Read ground truth RUL data to update the validation and test data.

In [226]:
val_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,condition
0,5001,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,1
1,5001,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,1
2,5001,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,1
3,5001,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,1
4,5001,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,1


## Data Preprocessing
First step is to generate labels for the training data which are Remaining Useful Life (RUL), label1 and label2 as was done in the [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3). Here, we will only make use of "label1" for binary clasification, while trying to answer the question: is a specific engine going to fail within w1 cycles?

In [227]:
def calc_training_rul(df):
    # Data Labeling - generate column RUL
    rul = pd.DataFrame(df.groupby('id')['cycle'].max()).reset_index()
    rul.columns = ['id', 'max']
    df = df.merge(rul, on=['id'], how='left')
    df['RUL'] = df['max'] - df['cycle']
    df.drop('max', axis=1, inplace=True)
    return df

In [228]:
train_df = calc_training_rul(train_df)
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,condition,RUL
0,1001,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,1,191
1,1001,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,1,190
2,1001,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,1,189
3,1001,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,1,188
4,1001,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,1,187


Calculating RUL for validation and test data is different. Both of these come for the test_FD\*.txt data set. These are associated with the actual RUL found in the corresponding RUL_FD\*.txt files. The last cycle in the these must equal the ground truth RUL. So we will work backwards from that.

In [229]:

val_rul_df = pd.read_csv(DATA_DIR+'RUL_FD001.txt', sep=" ", header=None)
# Remove the null column
val_rul_df.drop(val_rul_df.columns[[1]], axis=1, inplace=True)
val_rul_df.columns = ['RUL_actual']

print(val_rul_df.shape)
val_rul_df.head()


(100, 1)


,RUL_actual
0,112
1,98
2,69
3,82
4,91


In [230]:
# If index is not reset there will be int/str type issues when attempting the merge. 
val_cycle_count_df = val_df.groupby('id').count().reset_index()[['id','cycle']].rename(index=str, columns={"cycle":"cycles"}).reset_index(drop=True)
print(val_cycle_count_df.shape)

val_cycle_count_df.head()

(100, 2)


,id,cycles
0,5001,31
1,5002,49
2,5003,126
3,5004,106
4,5005,98


In [231]:
### Old
#df = val_cycle_count_df.reset_index(drop=True).merge(val_rul_df.reset_index(drop=True), left_index=True, right_index=True, how='left')
#df.head()

In [232]:
assert val_cycle_count_df.shape[0] == val_rul_df.shape[0]
df = val_cycle_count_df.merge(val_rul_df, left_index=True, right_index=True, how='left')
df.head()

,id,cycles,RUL_actual
0,5001,31,112
1,5002,49,98
2,5003,126,69
3,5004,106,82
4,5005,98,91


In [233]:
df['RUL_actual'] = df['cycles'] + df['RUL_actual']
df.drop('cycles',  axis=1, inplace=True)
df.head()

,id,RUL_actual
0,5001,143
1,5002,147
2,5003,195
3,5004,188
4,5005,189


In [234]:
val_df.dtypes[0] == df.dtypes[0]

True

In [235]:
val_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,condition
0,5001,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,1
1,5001,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,1
2,5001,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,1
3,5001,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,1
4,5001,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,1


In [236]:
# Join the two data frames
val_df = val_df.merge( df, on='id', how='left')
val_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,condition,RUL_actual
0,5001,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,1,143
1,5001,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,1,143
2,5001,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,1,143
3,5001,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,1,143
4,5001,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,1,143


In [237]:
val_df['RUL'] = val_df['RUL_actual'] - val_df['cycle']
val_df.drop('RUL_actual',  axis=1, inplace=True)
val_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,condition,RUL
0,5001,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,1,142
1,5001,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,1,141
2,5001,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,1,140
3,5001,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,1,139
4,5001,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,1,138


In [238]:
# Read in the val X data
val_df = load_data([DATA_DIR+'test_FD001.txt'], cols, sort_cols)

# Read in the val Y data.
val_rul_df = pd.read_csv(DATA_DIR+'RUL_FD001.txt', sep=" ", header=None)
# Remove the null column
val_rul_df.drop(val_rul_df.columns[[1]], axis=1, inplace=True)
val_rul_df.columns = ['RUL_actual']

# If index is not reset there will be int/str type issues when attempting the merge. 
val_cycle_count_df = val_df.groupby('id').count().reset_index()[['id','cycle']].rename(index=str, columns={"cycle":"cycles"}).reset_index(drop=True)

# For each engine, join cycle count and RUL actual
assert val_cycle_count_df.shape[0] == val_rul_df.shape[0]
df = val_cycle_count_df.merge(val_rul_df, left_index=True, right_index=True, how='left')
df['RUL_actual'] = df['cycles'] + df['RUL_actual']
df.drop('cycles',  axis=1, inplace=True)

# Join the two data frames
val_df = val_df.merge( df, on='id', how='left')

# Use the cycle to decrement the RUL until the ground truth is reached.
val_df['RUL'] = val_df['RUL_actual'] - val_df['cycle']
val_df.drop('RUL_actual',  axis=1, inplace=True)
val_df.head()


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,condition,RUL
0,5001,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,1,142
1,5001,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,1,141
2,5001,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,1,140
3,5001,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,1,139
4,5001,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,1,138


---

This section is when attempting binary classification. First define the window length w to represent the range from the end. Then add a new column called window to hold this. A value of 0 denotes healthy, 1, unhealthy.

In [239]:
w = 30
train_df['window'] = np.where(train_df['RUL'] <= w, 1, 0 )
train_df.head()

val_df['window'] = np.where(val_df['RUL'] <= w, 1, 0 )
val_df.head()


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,condition,RUL,window
0,5001,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,8.4052,0.03,392,2388,100.0,38.86,23.3735,1,142,0
1,5001,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,8.3803,0.03,393,2388,100.0,39.02,23.3916,1,141,0
2,5001,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,8.4441,0.03,393,2388,100.0,39.08,23.4166,1,140,0
3,5001,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,8.3917,0.03,391,2388,100.0,39.00,23.3737,1,139,0
4,5001,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,8.4031,0.03,390,2388,100.0,38.99,23.4130,1,138,0


### Scale the data

Apply MinMax scaler 

In [240]:
# Set up the columns that will be scaled
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL','window', 'condition'])

# The default activation function for LSTM tanh, so we'll use a range of [-1,1].
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df.head()

/home/saad/anaconda3/envs/tf/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,condition,RUL,window,cycle_norm
0,1001,1,-0.999619,-0.999525,1.0,1.0,0.939980,0.854585,0.804223,1.0,...,1.0,0.836735,1.0,1.0,0.944164,0.940747,1,191,0,-1.00000
1,1001,2,-0.999495,-0.999288,1.0,1.0,0.946000,0.865915,0.816384,1.0,...,1.0,0.836735,1.0,1.0,0.940128,0.941260,1,190,0,-0.99631
2,1001,3,-0.999791,-0.997864,1.0,1.0,0.949649,0.845447,0.821459,1.0,...,1.0,0.795918,1.0,1.0,0.936764,0.932408,1,189,0,-0.99262
3,1001,4,-0.999553,-0.998576,1.0,1.0,0.949649,0.817657,0.810304,1.0,...,1.0,0.836735,1.0,1.0,0.932055,0.935719,1,188,0,-0.98893
4,1001,5,-0.999676,-0.999051,1.0,1.0,0.950014,0.817978,0.831131,1.0,...,1.0,0.857143,1.0,1.0,0.933401,0.939119,1,187,0,-0.98524


In [241]:
val_df['cycle_norm'] = val_df['cycle']
norm_val_df = pd.DataFrame(min_max_scaler.transform(val_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=val_df.index)
val_join_df = val_df[val_df.columns.difference(cols_normalize)].join(norm_val_df)
val_df = val_join_df.reindex(columns = val_df.columns)
val_df = val_df.reset_index(drop=True)
val_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,condition,RUL,window,cycle_norm
0,5001,1,-0.999476,-0.997864,1.0,1.0,0.961872,0.831018,0.792780,1.0,...,1.0,0.836735,1.0,1.0,0.930710,0.935674,1,142,0,-1.00000
1,5001,2,-0.999714,-0.999288,1.0,1.0,0.937973,0.847905,0.779422,1.0,...,1.0,0.857143,1.0,1.0,0.941473,0.937692,1,141,0,-0.99631
2,5001,3,-0.999572,-0.998338,1.0,1.0,0.951656,0.839835,0.807766,1.0,...,1.0,0.857143,1.0,1.0,0.945510,0.940479,1,140,0,-0.99262
3,5001,4,-0.999386,-0.998576,1.0,1.0,0.951291,0.824765,0.832088,1.0,...,1.0,0.816327,1.0,1.0,0.940128,0.935697,1,139,0,-0.98893
4,5001,5,-0.999519,-0.998576,1.0,1.0,0.952568,0.841171,0.810543,1.0,...,1.0,0.795918,1.0,1.0,0.939455,0.940078,1,138,0,-0.98524


In [242]:
val_df.iloc[20:31, :]

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,condition,RUL,window,cycle_norm
20,5001,21,-0.999405,-0.999051,1.0,1.0,0.956034,0.820383,0.797520,1.0,...,1.0,0.836735,1.0,1.0,0.937437,0.938907,1,122,0,-0.926199
21,5001,22,-0.999529,-0.998338,1.0,1.0,0.951473,0.817604,0.820789,1.0,...,1.0,0.836735,1.0,1.0,0.936091,0.936065,1,121,0,-0.922509
22,5001,23,-0.999543,-0.998576,1.0,1.0,0.945453,0.842882,0.777842,1.0,...,1.0,0.857143,1.0,1.0,0.946855,0.929554,1,120,0,-0.918819
23,5001,24,-0.999614,-0.998813,1.0,1.0,0.949102,0.879115,0.802068,1.0,...,1.0,0.857143,1.0,1.0,0.936091,0.938305,1,119,0,-0.915129
24,5001,25,-0.999453,-0.999288,1.0,1.0,0.947825,0.815733,0.802451,1.0,...,1.0,0.857143,1.0,1.0,0.937437,0.936232,1,118,0,-0.911439
25,5001,26,-0.999362,-0.999763,1.0,1.0,0.952020,0.820276,0.839989,1.0,...,1.0,0.816327,1.0,1.0,0.924655,0.933690,1,117,0,-0.907749
26,5001,27,-0.999619,-0.998338,1.0,1.0,0.944723,0.838286,0.802834,1.0,...,1.0,0.836735,1.0,1.0,0.931382,0.937860,1,116,0,-0.904059
27,5001,28,-0.999481,-0.997389,1.0,1.0,0.941987,0.878901,0.807527,1.0,...,1.0,0.836735,1.0,1.0,0.928692,0.933077,1,115,0,-0.900369
28,5001,29,-0.999519,-0.998338,1.0,1.0,0.942352,0.840958,0.792301,1.0,...,1.0,0.836735,1.0,1.0,0.941473,0.934404,1,114,0,-0.896679
29,5001,30,-0.999705,-0.997626,1.0,1.0,0.957676,0.833316,0.805994,1.0,...,1.0,0.816327,1.0,1.0,0.946182,0.939398,1,113,0,-0.892989


## Model

[Keras LSTM](https://keras.io/layers/recurrent/) layers expect an input in the shape of a numpy array of 3 dimensions (samples, time steps, features) where samples is the number of training sequences, time steps is the look back window or sequence length and features is the number of features of each sequence at each time step. 

In [243]:
# Feature columns. 
feature_cols = ['cycle_norm', 'condition']

# Label columns
label_cols = ['RUL']

# Three operational setting columns
setting_cols = ['setting' + str(i) for i in range(1,4)]
feature_cols.extend(setting_cols)

# Twenty one sensor columns
sensor_cols = ['s' + str(i) for i in range(1,22)]
feature_cols.extend(sensor_cols)
len(feature_cols)

26

In [244]:
feature_cols

['cycle_norm',
 'condition',
 'setting1',
 'setting2',
 'setting3',
 's1',
 's2',
 's3',
 's4',
 's5',
 's6',
 's7',
 's8',
 's9',
 's10',
 's11',
 's12',
 's13',
 's14',
 's15',
 's16',
 's17',
 's18',
 's19',
 's20',
 's21']

In [245]:
label_cols

['RUL']

## LSTM Network
The first layer is an LSTM layer with 100 units followed by another LSTM layer with 50 units. Dropout is also applied after each LSTM layer to control overfitting. Final layer is a Dense output layer with single unit and sigmoid activation since this is a binary classification problem.

In [246]:
# Size of the series time window
sequence_length = 50

In [247]:
batch_size = 100


In [248]:
# Create the model
num_features = len(feature_cols)
num_out = len(label_cols)

model = Sequential()

#model.add(Masking(mask_value=0., input_shape=(sequence_length, num_features)))

model.add(LSTM(
         input_shape=(sequence_length, num_features),
         units=100,
         batch_input_shape=(batch_size, sequence_length, num_features),
         stateful=False,
         return_sequences=True))
#model.add(Dropout(0.2))

model.add(LSTM(
          units=50,
          return_sequences=False))
#model.add(Dropout(0.2))


model.add(Dense(units=num_out, activation='relu'))

opt = Adam(lr=1e-3, decay=0.0, amsgrad=True)
model.compile(loss=rmse, optimizer=opt, metrics=['mse', 'mae'])

In [249]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (100, 50, 100)            50800     
_________________________________________________________________
lstm_16 (LSTM)               (100, 50)                 30200     
_________________________________________________________________
dense_8 (Dense)              (100, 1)                  51        
Total params: 81,051
Trainable params: 81,051
Non-trainable params: 0
_________________________________________________________________
None


### Train

In [250]:
# Setup log directory
log_dir, checkpoint_path = set_log_dir(MODEL_DIR, "engine")

print("Log dir: ", log_dir)
print("Checkpoint path: ", checkpoint_path)

# Save the scaler for later use
from sklearn.externals import joblib 
joblib.dump(min_max_scaler, log_dir+'/engine_scaler.pkl') 

Log dir:  ./model/engine20181227T0114
Checkpoint path:  ./model/engine20181227T0114/engine_20181227T0114.h5


['./model/engine20181227T0114/engine_scaler.pkl']

In [251]:
training_runs = 3
initial_epoch = 0
    
tensorboard = TensorBoard(log_dir=log_dir,
                        histogram_freq=0, write_graph=True, write_images=False)

checkpointer = ModelCheckpoint(checkpoint_path, verbose=1, save_best_only=True)


for tr_run in range(training_runs):

    print("Training run: {}, epoch: {}".format(tr_run, initial_epoch))
    train_data_generator = TSDataGenerator(train_df, feature_cols, label_cols, batch_size=batch_size, num_steps=sequence_length, loop=True)
    train_data_generator.print_summary()

    val_data_generator = TSDataGenerator(val_df, feature_cols, label_cols, batch_size=batch_size, num_steps=sequence_length, loop=True)
    val_data_generator.print_summary()

    # Callbacks
    lr_decay = LRDecay(epochs_step=7)
    lrate = LearningRateScheduler(lr_decay.step_decay, verbose=1)
    
    earlystopper = EarlyStopping(patience=10, verbose=1)
    
    callbacks = [ tensorboard, checkpointer, lrate, earlystopper]

    # fit the network
    history = model.fit_generator(
        generator=train_data_generator.generate(), 
        validation_data=val_data_generator.generate(), 
        initial_epoch=initial_epoch,
        epochs=100, 
        steps_per_epoch=train_data_generator.summary()['max_steps'],
        validation_steps=val_data_generator.summary()['max_steps'],
        shuffle=False,
        verbose=1,
        callbacks=callbacks )
    
    # pick up after the last epoch
    initial_epoch = history.epoch[-1] + 1

Training run: 0, epoch: 0
Number of items:  709
Data shape:  (160359, 30)
Max Iterations:  90168
Max steps: 901 @ 100
Number of items:  100
Data shape:  (13096, 30)
Max Iterations:  3196
Max steps: 31 @ 100
Epoch 1/100

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
901/901 [==============================] - 101s 112ms/step - loss: 129.2056 - mean_squared_error: 19343.9997 - mean_absolute_error: 119.9073 - val_loss: 118.5042 - val_mean_squared_error: 15626.7038 - val_mean_absolute_error: 113.1836

Epoch 00001: val_loss improved from inf to 118.50421, saving model to ./model/engine20181227T0114/engine_20181227T0114.h5
Epoch 2/100

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
901/901 [==============================] - 99s 110ms/step - loss: 87.2946 - mean_squared_error: 10191.8417 - mean_absolute_error: 76.0332 - val_loss: 86.2234 - val_mean_squared_error: 8242.9636 - val_mean_absolute_error: 76.8448

Epoch 00002: val_loss improved from 118.50421

901/901 [==============================] - 100s 111ms/step - loss: 66.6790 - mean_squared_error: 6398.4079 - mean_absolute_error: 60.0465 - val_loss: 52.6067 - val_mean_squared_error: 3294.8441 - val_mean_absolute_error: 45.9336

Epoch 00021: val_loss did not improve from 50.69377
Epoch 22/100

Epoch 00022: LearningRateScheduler setting learning rate to 0.0001186.
901/901 [==============================] - 99s 110ms/step - loss: 61.8057 - mean_squared_error: 5239.4902 - mean_absolute_error: 55.1998 - val_loss: 52.4319 - val_mean_squared_error: 3264.4954 - val_mean_absolute_error: 46.2619

Epoch 00022: val_loss did not improve from 50.69377
Epoch 23/100

Epoch 00023: LearningRateScheduler setting learning rate to 0.0001186.
901/901 [==============================] - 100s 111ms/step - loss: 61.3707 - mean_squared_error: 5235.4104 - mean_absolute_error: 54.8864 - val_loss: 58.1726 - val_mean_squared_error: 4292.7601 - val_mean_absolute_error: 51.3014

Epoch 00023: val_loss did not improve

KeyboardInterrupt: 

In [ ]:
history.epoch[-1]

In [ ]:
try:
    model
except NameError:
    from keras.models import load_model
    
    model_path = MODEL_DIR+'engine_model.h5'
    print("Loading model: ", model_path)

    custom_objects={'rmse':rmse}
    model = load_model(model_path, custom_objects=custom_objects)
    model.summary()
else:
    print("using existing mode.")


In [ ]:
# Save the scaler for later use
try:
    min_max_scaler
except NameError:
    from sklearn.externals import joblib 
    
    scaler_path = MODEL_DIR+'/engine_scaler.pkl'
    print("Loading scaler: ", scaler_path)
    
    min_max_scaler = joblib.load(scaler_path) 
else:
    print("using existing min_max_scaler.")


In [ ]:
test_df = load_data([DATA_DIR+'test_FD002.txt'], cols, sort_cols)


In [ ]:
# Read in the X data
test_df = load_data([DATA_DIR+'test_FD002.txt'], cols, sort_cols)

# Read in the Y data.
test_rul_df = pd.read_csv(DATA_DIR+'RUL_FD002.txt', sep=" ", header=None)
# Remove the null column
test_rul_df.drop(test_rul_df.columns[[1]], axis=1, inplace=True)
test_rul_df.columns = ['RUL_actual']

# Calculate the number of cycles in the X data.
# If index is not reset there will be int/str type issues when attempting the merge. 
test_cycle_count_df = test_df.groupby('id').count().reset_index()[['id','cycle']].rename(index=str, columns={"cycle":"cycles"}).reset_index(drop=True)

# For each engine, join cycle count and RUL actual
assert test_cycle_count_df.shape[0] == test_rul_df.shape[0]
df = test_cycle_count_df.merge(test_rul_df, left_index=True, right_index=True, how='left')
df['RUL_actual'] = df['cycles'] + df['RUL_actual']
df.drop('cycles',  axis=1, inplace=True)

# Join the two data frames
test_df = test_df.merge( df, on='id', how='left')

# Use the cycle to decrement the RUL until the ground truth is reached.
test_df['RUL'] = test_df['RUL_actual'] - test_df['cycle']
test_df.drop('RUL_actual',  axis=1, inplace=True)
test_df.head()


In [ ]:
w = 30
test_df['window'] = np.where(test_df['RUL'] <= w, 1, 0 )
test_df.head()

In [ ]:

test_df['cycle_norm'] = test_df['cycle']
cols_normalize = test_df.columns.difference(['id','cycle','RUL','window', 'condition'])

norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)
test_df.head()

In [ ]:
test_df[ test_df['id']==6001 ].iloc[-50:]

In [ ]:
test_data_generator = TSDataGenerator(test_df, feature_cols, label_cols, batch_size=batch_size, num_steps=sequence_length, loop=False)
test_data_generator.print_summary()

X = []
y = []
for i, p in enumerate(test_data_generator.generate()):
    X.append(p[0])
    y.append(p[1])

test_X = np.vstack(X)
test_y = np.vstack(y)

In [ ]:
test_X.shape

In [ ]:
# training metrics
score = model.evaluate(test_X, test_y, verbose=1, batch_size=batch_size)
print('Test score:', score)


In [ ]:
t1_X = test_df[ test_df['id']==6004 ][feature_cols]
print(t1_X.head(5))
print(t1_X.shape)

t1_X = t1_X.iloc[:50].values
t1_X = t1_X.reshape((1, *t1_X.shape))
t1_X.shape

In [ ]:
t1_y = test_df[ test_df['id']==6004 ][label_cols]
print(t1_y.head(5))
print(t1_y.shape)

t1_y = t1_y.iloc[:50].values
t1_y.shape

Keras requires that you use same batch size in predicting as you used when training.

In [ ]:
inf_X = np.zeros((99,50,26))
inf_X = np.vstack((inf_X, t1_X))
print(inf_X.shape)


In [ ]:
y_pred_array = model.predict(inf_X, verbose=1, batch_size=batch_size)
y_pred = y_pred_array[-1:,0][0]
actual_rul = t1_y[0,0]
print("Pred={:.4f}, RUL={:.4f}, Diff={:.4f}".format( y_pred, actual_rul, actual_rul-y_pred))

In [ ]:
y_pred[-1:,0][0]

In [ ]:
# make predictions and compute confusion matrix
y_pred = model.predict(test_6010_X, verbose=1)
y_true = label_array
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true, y_pred)
cm

In [ ]:
# compute precision and recall
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print( 'precision = ', precision, '\n', 'recall = ', recall)

In [ ]:
d_6010 = test_data_generator.data[6010]

d_6010.reset()
test_batch = d_6010.get_batch(sequence_length)
test_6010_X = test_batch[feature_cols].values.reshape(1, 50, 26)
test_6010_y = test_batch[label_cols].values.reshape(1, 50, 1)

print("test_X: ", test_6010_X.shape)
print("test_y: ", test_6010_y.shape)

Next, we look at the performance on the test data. In the [Predictive Maintenance Template Step 1 of 3](https://gallery.cortanaintelligence.com/Experiment/Predictive-Maintenance-Step-1-of-3-data-preparation-and-feature-engineering-2), only the last cycle data for each engine id in the test data is kept for testing purposes. In order to compare the results to the template, we pick the last sequence for each id in the test data.

In [ ]:
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last.shape

In [ ]:
seq_array_test_last_df = [test_df[test_df['id']==id][sequence_cols] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

In [ ]:
seq_array_test_last_df

In [ ]:
seq_array_test_last_df[5].shape

In [ ]:
seq_array_test_last_df[5].tail()

In [ ]:
seq_array_test_last[5].shape

In [ ]:
seq_array_test_last[5][49,:]

In [ ]:
y_mask = [len(test_df[test_df['id']==id]) >= sequence_length for id in test_df['id'].unique()]

Similarly, we pick the labels.

In [ ]:
label_array_test_last = test_df.groupby('id')['label1'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
label_array_test_last.shape

In [ ]:
print(seq_array_test_last.shape)
print(label_array_test_last.shape)

In [ ]:
# test metrics
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose=2)
print('Accurracy: {}'.format(scores_test[1]))

In [ ]:
# make predictions and compute confusion matrix
y_pred_test = model.predict_classes(seq_array_test_last)
y_true_test = label_array_test_last
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true_test, y_pred_test)
cm

In [ ]:
# compute precision and recall
precision_test = precision_score(y_true_test, y_pred_test)
recall_test = recall_score(y_true_test, y_pred_test)
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
print( 'Precision: ', precision_test, '\n', 'Recall: ', recall_test,'\n', 'F1-score:', f1_test )

In [ ]:
results_df = pd.DataFrame([[scores_test[1],precision_test,recall_test,f1_test],
                          [0.94, 0.952381, 0.8, 0.869565]],
                         columns = ['Accuracy', 'Precision', 'Recall', 'F1-score'],
                         index = ['LSTM',
                                 'Template Best Model'])
results_df

Comparing the above test results to the predictive maintenance template, we see that the LSTM results are better than the template. It should be noted that the  data set used here is very small and deep learning models are known to perform superior with large datasets so for a more fair comparison larger datasets should be used.

## Future Directions and Improvements
This tutorial covers the basics of using deep learning in predictive maintenance and many predictive maintenance problems usually involve a variety of data sources that needs to be taken into account when applying deep learning in this domain. Additionally, it is important to tune the models for the right parameters such as window size. Here are some suggestions on future directions on improvements:
- Try different window sizes.
- Try different architectures with different number of layers and nodes.
- Try tuning hyperparmeters of the network.
- Try predicting RUL (regression) such as in  [Predictive Maintenance Template Step 2A of 3](https://gallery.cortanaintelligence.com/Experiment/Predictive-Maintenance-Step-2A-of-3-train-and-evaluate-regression-models-2) and label2 (multi-class classification) such as in [Predictive Maintenance Template Step 2C of 3](https://gallery.cortanaintelligence.com/Experiment/Predictive-Maintenance-Step-2C-of-3-train-and-evaluation-multi-class-classification-models-2).
- Try on larger data sets with more records.
- Try a different problem scenario such as in [Predictive Maintenance Modelling Guide](https://gallery.cortanaintelligence.com/Notebook/Predictive-Maintenance-Modelling-Guide-R-Notebook-1) where multiple other data sources are involved such as maintenance records.

